In [1]:
from pyBind import py_FEA, py_Sensitivity
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse.linalg

import sys
sys.path.insert(0, r'/home/hac210/Dropbox/packages/topOpt_MDO/lib/fem2d/fem2d/')
from utils.plot import get_mesh, plot_solution, plot_contour
from fem2d import PyFEMSolver

sys.path.insert(0, r'/home/hac210/Dropbox/packages/02.M2DO_opensource_new/OpenLSTO/M2DO_FEA/Python/Cant')
from optim_refact_v3 import Cantilever

In [2]:
# FEM setup
num_nodes_x = 21 # 21
num_nodes_y = 41 #11

length_x = 160.0
length_y = 80.0

num_elems_x = num_nodes_x - 1
num_elems_y = num_nodes_y - 1

num_elem = num_elems_x * num_elems_y
num_node = num_nodes_y * num_nodes_x

E = 1.0
v = 0.3

In [3]:
# get Cantil + compte elasticity =====================================
cant = Cantilever(isHoles=False, lxy=[length_x,length_y], exy=[num_elems_x,num_elems_y])
cant.set_fea()
u_cant = cant.get_u(np.ones(num_elem), f=-1.0)

[440]


/home/hac210/anaconda2/lib/python2.7/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [4]:
# get fem2d + compute elasticity =====================================
fem_solver = PyFEMSolver(num_nodes_x,num_nodes_y,length_x,length_y,E,v)
force = np.zeros(2*num_node)
force[(num_nodes_y*(num_nodes_x-1) + int(num_nodes_y/2))*2+1] = -1.0
nodes = get_mesh(num_nodes_x, num_nodes_y, length_x, length_y) 

# stiffness matrix
size = num_elem * 64 * 4 + 2 * 2 * num_nodes_y
state_size = 2 * num_node + 2 * num_nodes_y
rhs = np.zeros(state_size)
rhs[:2*num_node] = force

data1 = np.zeros(size)
rows1 = np.zeros(size, np.int32)
cols1 = np.zeros(size, np.int32)
fem_solver.get_stiffness_matrix(np.ones(num_node), data1, rows1, cols1)
mtx = scipy.sparse.csc_matrix((data1, (rows1, cols1)), shape=(state_size, state_size))

print(data1.shape, rows1.shape, cols1.shape, state_size)

sol = scipy.sparse.linalg.spsolve(mtx, rhs)

((204964,), (204964,), (204964,), 1804)


In [5]:
# get py_FEA + compute elasticity =====================================
pyFEA = py_FEA(lx = length_x, ly = length_y, nelx = num_elems_x, nely = num_elems_y, element_order = 2)
[node, elem, elem_dof] = pyFEA.get_mesh()
pyFEA.set_material(E,v,1.0)

coord = np.array([0,0])
tol = np.array([1e-3,1e10])
pyFEA.set_boundary(coord = coord,tol = tol)
BCid = pyFEA.get_boundary()

coord = np.array([length_x,length_y/2])
tol = np.array([1,1]) 
GF_ = pyFEA.set_force(coord = coord,tol = tol, direction = 1, f = -1.0) 

# quickfix (for comparison)
(rows, cols, vals) = pyFEA.compute_K()
u = pyFEA.solve_FE()

# using Lagrange multiplier
nprows = np.array(rows)
npcols = np.array(cols)
npvals = np.array(vals)

nDOF  = num_node * 2
nDOF_withLag  = num_node * 2 + len(BCid)

mtx = scipy.sparse.csc_matrix((npvals, (nprows, npcols)), shape = (nDOF_withLag, nDOF_withLag))

GF = np.zeros(nDOF_withLag)
GF[:nDOF] = GF_ #rhs.transpose().flatten(order = 'F')

sol_lsto = scipy.sparse.linalg.spsolve(mtx, GF) # direct solver is used

# print npcols.shape
# print nDOF

# mtx = scipy.sparse.csc_matrix((npvals, (nprows, npcols)), shape = (nDOF, nDOF))

# # force 
# rhs = np.zeros((num_node,2))

# fixedNodes = (abs(node[:,0]-coord[0]) < tol[0]) & (abs(node[:,1]-coord[1]) < tol[1])
# nid = np.where(fixedNodes == True)[0]



# rhs[nid,1] = -1
# GF = rhs.transpose().flatten(order = 'F')
# state = scipy.sparse.linalg.spsolve(mtx, la) # direct solver is used

In [6]:
print "cant = " + str(min(u_cant[0]))
print "pyFEA = " + str(min(u))
print "pyFEA_lag = " + str(min(sol_lsto))
print "fem2d = " + str(min(sol))

cant = -38.786160943
pyFEA = -38.7861598793
pyFEA_lag = -38.7861609429
fem2d = -38.7861609431


In [7]:
# changing of the the areafraction
areafraction =  np.random.rand(num_elem)

## CANTILEVER VER. (correct for SIMP)
u_cant = cant.get_u(areafraction,-1)

## OPENLSTO VER
(rows, cols, vals) = pyFEA.compute_K_SIMP(areafraction)

### quickfix (for comparison)
u = pyFEA.solve_FE()

### using Lagrange multiplier
nprows = np.array(rows)
npcols = np.array(cols)
npvals = np.array(vals)

nDOF  = num_node * 2
nDOF_withLag  = num_node * 2 + len(BCid)

mtx = scipy.sparse.csc_matrix((npvals, (nprows, npcols)), shape = (nDOF_withLag, nDOF_withLag))

GF = np.zeros(nDOF_withLag)
GF[:nDOF] = GF_ #rhs.transpose().flatten(order = 'F')

sol_lsto = scipy.sparse.linalg.spsolve(mtx, GF) # direct solver is used

## FEM2D VER
data1 = np.zeros(size)
rows1 = np.zeros(size, np.int32)
cols1 = np.zeros(size, np.int32)

fem_solver.set_area_fractions(areafraction)
fem_solver.get_stiffness_matrix_SIMP(data1, rows1, cols1)
mtx = scipy.sparse.csc_matrix((data1, (rows1, cols1)), shape=(state_size, state_size))

sol = scipy.sparse.linalg.spsolve(mtx, rhs)

In [8]:
print "cant = " + str(min(u_cant[0]))
print "pyFEA = " + str(min(u))
print "pyFEA_lag = " + str(min(sol_lsto[:nDOF]))
print "fem2d = " + str(min(sol[:nDOF]))

cant = -92.4024924539
pyFEA = -92.402489345
pyFEA_lag = -92.4024924539
fem2d = -92.402492454


In [ ]:
# sensitivity calculation (SIMP)
